# COMPSCI 589 HW1

#### Name: Haochen Wang

### SECTION 0: Load Libraries

In [1027]:
import sklearn.model_selection
import scipy
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
from operator import itemgetter
from collections import Counter

### SECTION 2: Evaluating Decision Tree

In [1028]:
# Import Document

house_file = open('house_votes_84.csv', encoding='utf-8-sig')
csvreader = csv.reader(house_file)
houserows = []
for row in csvreader:
        houserows.append(row)

hvcat = houserows[0]
# print(housevotecat)

housevotedat = np.array([[int(element) for element in row] for row in houserows[1:]])

#print(len(housevotedat))

In [1029]:
# Node Class
class Treenode:
    label = ""
    type = ""
    subattributelist = []
    

    def __init__(self, label, type):
        self.label = label
        self.type = type


In [1030]:
# Define helper functions that I use in decision tree.

def same(column):
    return all(item == column[0] for item in column)

def majority(column):
    return np.argmax(np.bincount(column))

def entropy(col):
    values = list(Counter(col).values())
    ent = 0
    for value in values:
        k = (value/sum(values))
        ent += -k*math.log(k,2)
    return ent

# test = np.array([1,1,3,3,11,11])
# same(test)
# majority(test)

# test =  np.array([1,1,0,0,0])
# cc = dict(Counter(test))
# dd = list(cc.values())
# entropy(test)

In [1031]:
# Define three test criteria: 
    
# ID3 - Entropy Gain 

def id3gain(collist, listattribution):
    original_ent = entropy(collist[-1])
    smallest_ent = 1
    i = 0
    # bestindex = i
    best = listattribution[i]
    for attributes in listattribution[:-1]: # I keep the last column: the target/label.
        liskey = list(Counter(collist[i]).keys())
        listofcategory = []
        for value in liskey:
            index = [idx for idx, element in enumerate(collist[i]) if element == value]
            category = np.array(collist[-1][index]) 
            listofcategory.append(category) # list of nparrays of target/label/categories.

        ent = 0
        for cat in listofcategory:
            a = len(cat)/len(collist[i]) # This is probability
            ent += a * entropy(cat) # probability multiple by entropy

        if ent < smallest_ent:
            smallest_ent = ent
            best = attributes
            # bestindex = i
        i+=1

    return best, original_ent-ent

# C4.5 - Entrophy Ratio 
# CART - Gini Impurity


weather =  np.array([0,0,1,2,2,2,1,0,0,2,0,1,1,2])
windy = np.array([0,1,0,0,0,1,1,0,0,0,1,1,0,1])
yesno = np.array([0,0,1,1,1,0,1,0,1,1,1,1,1,0])
testdata = np.array((windy,weather,yesno))
listattri = ["windy","weather","yesno"]

id3gain(testdata,listattri)

('weather', 0.2467498197744391)

In [ ]:
#test module

weather =  np.array([0,0,1,2,2,2,1,0,0,2,0,1,1,2])
windy = np.array([0,1,0,0,0,1,1,0,0,0,1,1,0,1])
yesno = np.array([0,0,1,1,1,0,1,0,1,1,1,1,1,0])
testdata = np.array((windy,weather,yesno))
listattri = ["windy","weather","yesno"]

id3gain(testdata,listattri)

In [1033]:
# Decision Tree
def decisiontree(dataset, listattributes, type='id3'):
    if type == "id3":
        criteria = 1
    elif type == "c4.5":
        criteria = 2
    elif type == "cart":
        criteria = 3

    node = Treenode("-1","node")

    if same([row[-1] for row in dataset]):
        node.type = "leaf"
        node.label = row[-1]
        return node
    
    if len(listattributes) == 0:
        node.type = "leaf"
        node.label = majority(dataset.T[-1])
        return node

    return